In [2]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
import re
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [3]:
import pandas as pd

# Charger le fichier CSV
df = pd.read_csv('dataset/phrases_emojis.csv')

# Liste pour stocker les nouvelles lignes de données
new_data = []

# Itérer sur chaque ligne du DataFrame
for _, row in df.iterrows():
    phrase = row['Phrase']
    emojis = row['Emojis']
    
    # Séparer les emojis par un espace ou un autre séparateur
    emoji_list = list(emojis)  # Cela sépare chaque emoji dans une liste

    # Ajouter une nouvelle ligne pour chaque emoji
    for emoji in emoji_list:
        if emoji != "" and emoji!= " " and emoji!= '️' and emoji !=' ' and emoji != '' and emoji != '\u200d' and emoji != '♂' and emoji != '♀' and emoji != '\uFE0F' :  # Vérifier que l'emoji n'est pas vide
            new_data.append({"Phrase": phrase, "Emojis": emoji})

# Créer un nouveau DataFrame avec les données traitées
new_df = pd.DataFrame(new_data)

# Supprimer les doublons (phrase, emoji identiques)
new_df = new_df.drop_duplicates()

# Sauvegarder le résultat dans un nouveau fichier CSV
new_df.to_csv('dataset/fichier_transforme.csv', index=False)



In [4]:
# Fonction de prétraitement du texte
def preprocess_text(text):
    text = text.lower()  # Convertir en minuscule
    text = re.sub(r'http\S+', '', text)  # Supprimer les liens
    text = re.sub(r'@\w+', '', text)  # Supprimer les mentions
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Supprimer les caractères non-alphabétiques
    return text

new_df = pd.read_csv('dataset/fichier_transforme.csv')

# Prétraitement
new_df['tweet_processed'] = new_df['Phrase'].apply(preprocess_text)

In [6]:
# Tokenisation des tweets
tokenized_tweets = [word_tokenize(tweet,language="french") for tweet in new_df['tweet_processed']]

# Entraîner un modèle Word2Vec
model = Word2Vec(sentences=tokenized_tweets, window=2, min_count=1, sg=1,workers=4)

# Sauvegarder le modèle
model.save("models/word2vec_model_twitter.model")
print("Modèle sauvegardé.")

/home/rb/Emoji-Prediction/.venv/lib/python3.12/site-packages/gensim/models/base_any2vec.py:617: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  warnings.warn(


Modèle sauvegardé.


In [7]:
# Fonction pour obtenir l'embedding moyen d'un tweet
def get_tweet_embedding(tweet, model):
    tokens = word_tokenize(preprocess_text(tweet))  # Tokenisation
    embeddings = []
    
    for word in tokens:
        if word in model.wv:
            embeddings.append(model.wv[word])
    
    if embeddings:
        return np.mean(embeddings, axis=0)  # Renvoie l'embedding moyen
    else:
        return np.zeros(model.vector_size)  # Si aucun mot trouvé dans le modèle, renvoie un vecteur nul

# Extraire les embeddings des tweets
X = np.array([get_tweet_embedding(tweet, model) for tweet in new_df['Phrase']])

# Encoder les emojis en labels numériques
encoder = LabelEncoder()
new_df['emoji_label'] = encoder.fit_transform(new_df['Emojis'])

# Définir les labels (y) et les caractéristiques (X)
y = new_df['emoji_label'].values

# Générer les indices d'origine
indices = np.arange(len(X))

# Diviser les données en ensemble d'entraînement, de test et les indices correspondants
X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(
    X, y, indices, test_size=0.2, random_state=42
)


clf =  RandomForestClassifier(n_estimators=200)
clf.fit(X_train, y_train)


RandomForestClassifier(n_estimators=200)

In [8]:
# Prédiction sur l'ensemble de test
y_pred = clf.predict(X_test)

# Évaluation de la précision du modèle
accuracy = accuracy_score(y_test, y_pred)
print(f"Précision du modèle : {accuracy * 100:.2f}%")

# Afficher des emojis prédits pour quelques exemples
predicted_emojis = encoder.inverse_transform(y_pred)


# Récupérer les tweets et emojis réels correspondants aux indices de l'ensemble de test
tweets_predicted = new_df.iloc[test_indices]['Phrase']
emojis_real = new_df.iloc[test_indices]['Emojis']

# Afficher les résultats : tweet, emoji réel et emoji prédit
for tweet, emoji_real, emoji_pred in zip(tweets_predicted.head(30), emojis_real.head(30), predicted_emojis[:30]):
    print(f"Tweet: {tweet}\nEmoji réel: {emoji_real}\nEmoji prédit: {emoji_pred}\n")
    




Précision du modèle : 4.42%
Tweet: faire découvrir ce vaste univers
Emoji réel: 🪐
Emoji prédit: 😂

Tweet: Recette de pain maison
Emoji réel: 🍞
Emoji prédit: 🗽

Tweet: ️ Sébastien Haller rejoint le FC Utrecht en prêt du Borussia Dortmund,
Emoji réel: 🤝
Emoji prédit: ⚪

Tweet: le Toit de l'Indochine INRATABLE
Emoji réel: 🌄
Emoji prédit: 👾

Tweet: Bonjour la France
Emoji réel: 😜
Emoji prédit: ❤

Tweet: Bon petit mood tristou de la mi-semaine !  (J'ai regardé la fin de Charmed et maintenant je suis comme une madeleine !). La scène de fin pour les vrais qui connaissent, m'a fait me questionner sur ma vie (futur).
Emoji réel: 😭
Emoji prédit: 😅

Tweet: des surgelés: Du cordon Blö
Emoji réel: 🤣
Emoji prédit: 📚

Tweet: C’est bien la piste Mddr
Emoji réel: 😂
Emoji prédit: 😅

Tweet: ️ Dernier jour de tournage
Emoji réel: 🕊
Emoji prédit: 🎥

Tweet: pour nager sans se cogner
Emoji réel: ✊
Emoji prédit: 😭

Tweet: L'année 2024
Emoji réel: 🤯
Emoji prédit: 👀

Tweet: offrent pas mal de missions ️
Emoji r

In [9]:
phrase_list = ["eh le prof de compilation là avec son projet de con"]
for phrase in phrase_list :
    x_sample=get_tweet_embedding(phrase, model)
    y_sample=clf.predict([x_sample])

    predicted= encoder.inverse_transform(y_sample)
    print(phrase, predicted)

eh le prof de compilation là avec son projet de con ['😂']


In [21]:
from sklearn.manifold import TSNE
import plotly.express as px
import numpy as np

# Étape 1 : Prédire les emojis pour chaque tweet
y_pred = clf.predict(X)  # Prédictions des emojis
predicted_emojis = encoder.inverse_transform(y_pred)  # Mapper les indices aux emojis

# Ajouter les prédictions au DataFrame
new_df['emoji_pred'] = predicted_emojis

# Étape 2 : Calculer l'embedding moyen par emoji prédit
emoji_embeddings = []
emoji_labels = []

for emoji in np.unique(predicted_emojis):
    # Filtrer les tweets prédits avec cet emoji
    tweets_with_emoji = X[y_pred == encoder.transform([emoji])[0]]
    
    # Calculer l'embedding moyen
    if len(tweets_with_emoji) > 0:
        mean_embedding = np.mean(tweets_with_emoji, axis=0)
        emoji_embeddings.append(mean_embedding)
        emoji_labels.append(emoji)

emoji_embeddings = np.array(emoji_embeddings)

# Étape 3 : Réduction de dimension avec t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=5, n_iter=1000)
tsne_results = tsne.fit_transform(emoji_embeddings)

# Étape 4 : Créer un DataFrame pour la visualisation
tsne_df = pd.DataFrame({
    'tsne_1': tsne_results[:, 0],
    'tsne_2': tsne_results[:, 1],
    'emoji': emoji_labels
})

# Étape 5 : Visualisation avec Plotly
fig = px.scatter(
    tsne_df,
    x='tsne_1',
    y='tsne_2',
    text='emoji',  # Afficher les emojis sur le graphe
    title="Représentation t-SNE des embeddings moyens par emoji prédit",
    labels={'emoji': 'Emoji'},
        width=1500,  # Largeur du graphique
    height=1500 
)

fig.update_traces(
    textposition='top center', 
    textfont=dict(size=30),  # Taille du texte
    marker=dict(size=20)  # Taille des points
)

# Afficher le graphique
fig.show()


/home/rb/Emoji-Prediction/.venv/lib/python3.12/site-packages/sklearn/manifold/_t_sne.py:1164: FutureWarning:

'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.

